# Investigating different penalty terms

## Introduction 

We apply our model in a real data example with two real measurement instruments, i.e., without artificial modifications, where discrepancies between measurement instruments could arise due to more complex reasons. We consider data from the HFMSE and RULM measurement instruments in the SMArtCARE data as described in Section 3.1 in the manuscript.

On this dataset, we compare different versions of penalties in our approach with respect to their effectiveness in achieving a close alignment, as described in Section 3.4 in the manuscript. 
Specifically, we train the model with four different loss function versions, using 
 1. neither the adversarial penalty term nor the ODE penalty,
 2. only the ODE penalty, i.e., $\alpha=0, \beta \neq 0$,
 3. only the adversarial penalty, i.e., $\alpha \neq 0, \beta=0$, 
 4. both penalty terms, i.e., $\alpha, \beta \neq 0$.

## Setup

We start by loading the Julia environment, the necessary package libraries and functions from the `src` directory that we will use in this notebook.

## Julia environment

In [ ]:
cd(@__DIR__)

using Pkg;
Pkg.activate("../.")
Pkg.status()

## Package libraries

In [ ]:
using CSV
using Dates
using DataFrames
using Distributions
using Random
using GLM
using Flux
using LaTeXStrings
using LinearAlgebra
using Measures
using Parameters
using Plots 
using ProgressMeter
using StatsBase
gr() # setting the plotting backend

## Loading functions

In [ ]:
sourcedir = "../src/"
include(joinpath(sourcedir, "load_data.jl"))
include(joinpath(sourcedir, "model.jl"))
include(joinpath(sourcedir, "ODE_solutions.jl"))
include(joinpath(sourcedir, "training.jl"))
include(joinpath(sourcedir, "eval_penalties.jl"))
include(joinpath(sourcedir, "plot_latent.jl"))

## Loading and preparing data 

We load the baseline information and the timedependent data separately. We subset to the RULM and HFMSE measurements, respectively, then prepare the data for the training process (for details, see the corresponding functions or the description in Section 3.1 in the manuscript).

In [ ]:
data_path = joinpath("../dataset/")

baseline_df = CSV.File(string(data_path, "baseline_df.csv"), truestrings = ["TRUE", "M"], falsestrings = ["FALSE", "F"], missingstring = ["NA"], decimal=',') |> DataFrame
timedepend_df = CSV.File(string(data_path, "timedepend_df.csv"), truestrings = ["TRUE"], falsestrings = ["FALSE"], missingstring = ["NA"], decimal=',') |> DataFrame

# remove "Column1"
baseline_df = baseline_df[:,2:end]
timedepend_df = timedepend_df[:,2:end]

other_vars = ["patient_id", "months_since_1st_test", "feeding_tube", 
            "scoliosis_yn", "pain_yn", "fatigue_yn", "ventilation", "adverse_event", 
            "fvc_yn", "fvc_percent",
            "gen_impr", "mf_impr", "rf_impr"
]

baseline_vars = names(baseline_df)[findall(x -> !(x ∈ ["cohort", "baseline_date"]), names(baseline_df))]

test1="hfmse"
test2="rulm"
mixeddata = get_data_tests(timedepend_df, baseline_df, 
    other_vars, baseline_vars; 
    test1=test1, test2=test2, remove_lessthan1=true
);

Finally, we set up the directory structure for saving the reults:

In [ ]:
!isdir("../results") && mkdir("../results")
save_eval_dir = "../results/penalties"
!isdir(save_eval_dir) && mkdir(save_eval_dir)

## Define the models

We set up the models by defining the ODE dynamics and define two model configurations, `modelargs1` and `modelargs2`, for the respective VAE of each measurement instrument. Specifically, `p` corresponds to the number of time-depedenent variables of each instruments, i.e., the number of test items, and `q` to the number of baseline variables. The `dynamics` are set to the previously defined ODE dynamics. 
Both models have a `seed` value for reproducibility. 
The last settings concern the setup of the additional neural network that maps the baseline variables to a set of individual-specific ODE parameters (see the implementation details in the Appendix of the manuscript or the definition of the `odevae` struct in `model.jl` for details).

In [ ]:
nODEparams = 6
dynamics = params_fullinhomogeneous

# init models
modelargs1 = ModelArgs(p=size(mixeddata.xs1[1],1), 
                    q=length(mixeddata.xs_baseline[1]), 
                    dynamics=dynamics,
                    bottleneck=false,
                    seed=1234,
                    scale_sigmoid=2, 
                    add_diagonal=true 
)
modelargs2 = ModelArgs(p=size(mixeddata.xs2[1],1), 
                    q=length(mixeddata.xs_baseline[1]), 
                    dynamics=dynamics,
                    bottleneck=false,
                    seed=1234,
                    scale_sigmoid=2, 
                    add_diagonal=true 
)


Next, we initialize the models and set up the training configurations:

In [ ]:
m1 = odevae(modelargs1);
m2 = odevae(modelargs2);

## Training the model

We then set parameters for the loss function, specifically the weights of the different penalties. In this first illustration, we will use both the ODE (`λ_μpenalty`) and the adversarial penalty (`λ_adversarialpenalty`) (see the Methods Section of the manuscript for more explanation on the loss function and its terms). 
These parameters are used to create a `LossArgs` object containing the parameters that control the loss function. 

We then define the training hyperparameters in the `TrainingArgs` struct, controlling the number of training epochs and the learning rate. 

In [ ]:
# loss args
args_joint=LossArgs(
    λ_μpenalty=5.0f0,
    λ_adversarialpenalty=5.0f0,
    λ_variancepenalty=5.0,
    variancepenaltytype = :sum_ratio,
    variancepenaltyoffset = 1.0f0, 
    skipt0=true,
    weighting=true, 
)

trainingargs_joint=TrainingArgs(warmup=false, epochs=10, lr=0.00003)

Now, we're ready to train the model. 

By setting `verbose=true`, we can print out the loss function value after each training epoch. 

By setting `plotting=true`, we can additionally plot the latent fits of a few randomly selected individuals after each training epoch.

In [ ]:
train_mixed_model!(m1, m2, mixeddata, args_joint, trainingargs_joint, 
    verbose=true, plotting=true)

## Comparing different penalty versions

In the following, we now train the model four times, each time with a different configuration of the penalty term weights. We save the models and subsequently evaluate the model predictions and compare to a simple baseline method. We also plot the fitted latent trajectories of a few randomly selected individuals for each model configuration and save the results.

This reproduces the results shown in Figure 4 and Table in Section 3.4 in the manuscript. 

### Neither penalty

We re-initialize the models, define the corresponding loss function arguments, train and save the models.

In [ ]:
m1 = odevae(modelargs1);
m2 = odevae(modelargs2);

# loss args
args_joint=LossArgs(
    λ_μpenalty=0.0f0,
    λ_adversarialpenalty=0.0f0,
    λ_variancepenalty=5.0,
    variancepenaltytype = :sum_ratio,
    variancepenaltyoffset = 1.0f0, 
    skipt0=true,
    weighting=true, 
)
# prepare training
trainingargs_joint=TrainingArgs(warmup=false, epochs=10, lr=0.00003)
# train 
train_mixed_model!(m1, m2, mixeddata, args_joint, trainingargs_joint, verbose=false, plotting=false)
# save trained models 
no_penalty_models = Dict("m1" => deepcopy(m1), "m2" => deepcopy(m2))

To evaluate the models, we first set up an empty `DataFrame`in which we collect the prediction errors of the ODE and a simple linear baseline model for each penalty configuration.

In [ ]:
all_prederrs_df = DataFrame(
    PenaltyType = [],
    Dimension = [],
    ODE = [],
    LinearModel = []
)

Now, we evaluate the model predictions, calculate the baseline model and append to the overall dataframe.

In [ ]:
df_all, ODEprederrs1, ODEprederrs2 = 
    eval_prediction(no_penalty_models["m1"], no_penalty_models["m2"], 
    mixeddata, args_joint, 
    verbose=false
)

# baseline linear model
prederrdf = fit_baseline_model(df_all, modelargs1.q; verbose=false)

# append to overall df 
prederrdf[!, :PenaltyType] .= "No penalty"
append!(all_prederrs_df, prederrdf)

We additionally save exemplary plots of the fitted latent trajectories of a few randomly selected individuals.

First, we set up the directory for saving the plots and define the selection of individuals.

In [ ]:
# get selection of individuals 
Random.seed!(789)
selected_ids = rand(mixeddata.ids, 12)

# make corresponding directory
save_plots_dir = save_eval_dir
!isdir(save_plots_dir) && mkdir(save_plots_dir)

Now, we generate and save the plot.

In [ ]:
plot_no_penalty = plot(
    plot_selected_ids_final(no_penalty_models["m1"], no_penalty_models["m2"], 
        mixeddata, args_joint, selected_ids, 
        colors_points = ["#3182bd" "#9ecae1"; "#e6550d" "#fdae6b"], 
        marker_sizes = [7, 5]
    ), 
    plot_title="No ODE or adversarial penalty"
)
savefig(plot_no_penalty, joinpath(save_plots_dir, "no_penalty.pdf"))
plot_no_penalty

### Only ODE penalty

We re-initialize the models, define the corresponding loss function arguments, train and save the models.

In [ ]:
m1 = odevae(modelargs1);
m2 = odevae(modelargs2);

# loss args
args_joint=LossArgs(
    λ_μpenalty=5.0f0,
    λ_adversarialpenalty=0.0f0,
    λ_variancepenalty=5.0,
    variancepenaltytype = :sum_ratio,
    variancepenaltyoffset = 1.0f0, 
    skipt0=true,
    weighting=true, 
)
# prepare training
trainingargs_joint=TrainingArgs(warmup=false, epochs=10, lr=0.00003)
# train 
train_mixed_model!(m1, m2, mixeddata, args_joint, trainingargs_joint, verbose=false, plotting=false)

only_ODE_penalty_models = Dict("m1" => deepcopy(m1), "m2" => deepcopy(m2))


Now, we evaluate the model predictions, calculate the baseline model and append to the overall dataframe.

In [ ]:
df_all, ODEprederrs1, ODEprederrs2 = 
    eval_prediction(only_ODE_penalty_models["m1"], only_ODE_penalty_models["m2"], 
        mixeddata, args_joint, 
        verbose=false
)

# baseline model
prederrdf = fit_baseline_model(df_all, modelargs1.q; verbose=false)

# append to overall df
prederrdf[!, :PenaltyType] .= "Only ODE penalty"
append!(all_prederrs_df, prederrdf)

Finally, we generate and save the plot of the fitted latent trajectories of  selected individuals.

In [ ]:
plot_only_ODE_penalty = plot(
    plot_selected_ids_final(only_ODE_penalty_models["m1"], 
        only_ODE_penalty_models["m2"], 
        mixeddata, args_joint, selected_ids, 
        colors_points = ["#3182bd" "#9ecae1"; "#e6550d" "#fdae6b"], 
        marker_sizes = [7, 5]
    ), 
    plot_title="Only ODE penalty"
)
savefig(plot_only_ODE_penalty, joinpath(save_plots_dir, "only_ODE_penalty.pdf"))
plot_only_ODE_penalty

### Only adversarial penalty

We re-initialize the models, define the corresponding loss function arguments, train and save the models.

In [ ]:
m1 = odevae(modelargs1);
m2 = odevae(modelargs2);

# loss args
args_joint=LossArgs(
    λ_μpenalty=0.0f0,# 1.0f0
    λ_adversarialpenalty=5.0f0,#1.0f0,
    λ_variancepenalty=5.0,
    variancepenaltytype = :sum_ratio,
    variancepenaltyoffset = 1.0f0, 
    skipt0=true,
    weighting=true, 
)
# prepare training
trainingargs_joint=TrainingArgs(warmup=false, epochs=10, lr=0.00003)# lr=0.00008
# train 
train_mixed_model!(m1, m2, mixeddata, args_joint, trainingargs_joint, verbose=false, plotting=false)

only_adversarial_penalty_models = Dict("m1" => deepcopy(m1), "m2" => deepcopy(m2))


Now, we evaluate the model predictions, calculate the baseline model and append to the overall dataframe.

In [ ]:
df_all, ODEprederrs1, ODEprederrs2 = 
    eval_prediction(only_adversarial_penalty_models["m1"], 
        only_adversarial_penalty_models["m2"], 
        mixeddata, args_joint, 
        verbose=false
)

# baseline model
prederrdf = fit_baseline_model(df_all, modelargs1.q; verbose=false)
prederrdf[!, :PenaltyType] .= "Only adversarial penalty"
append!(all_prederrs_df, prederrdf)

Finally, we generate and save the plot of the fitted latent trajectories of  selected individuals.

In [ ]:
plot_only_adversarial_penalty = plot(
    plot_selected_ids_final(
        only_adversarial_penalty_models["m1"], only_adversarial_penalty_models["m2"], 
        mixeddata, args_joint, selected_ids, 
        colors_points = ["#3182bd" "#9ecae1"; "#e6550d" "#fdae6b"], 
        marker_sizes = [7, 5]
    ), 
    plot_title="Only adversarial penalty"
)
savefig(plot_only_adversarial_penalty, joinpath(save_plots_dir, "only_adversarial_penalty.pdf"))
plot_only_adversarial_penalty

## Both penalties

We re-initialize the models, define the corresponding loss function arguments, train and save the models.

In [ ]:
m1 = odevae(modelargs1);
m2 = odevae(modelargs2);

# loss args
args_joint=LossArgs(
    λ_μpenalty=5.0f0,# 1.0f0
    λ_adversarialpenalty=5.0f0,#1.0f0,
    λ_variancepenalty=5.0,
    variancepenaltytype = :sum_ratio,
    variancepenaltyoffset = 1.0f0, 
    skipt0=true,
    weighting=true, 
)
# prepare training
trainingargs_joint=TrainingArgs(warmup=false, epochs=10, lr=0.00003)# lr=0.00008
# train 
train_mixed_model!(m1, m2, mixeddata, args_joint, trainingargs_joint, verbose=false, plotting=false)

ODE_and_adversarial_penalty_models = Dict("m1" => deepcopy(m1), "m2" => deepcopy(m2))

Now, we evaluate the model predictions, calculate the baseline model and append to the overall dataframe.

In [ ]:
df_all, ODEprederrs1, ODEprederrs2 = 
    eval_prediction(ODE_and_adversarial_penalty_models["m1"], 
    ODE_and_adversarial_penalty_models["m2"], 
    mixeddata, args_joint, 
    verbose=false
)

# baseline model
prederrdf = fit_baseline_model(df_all, modelargs1.q; verbose=false)

# append to overall df
prederrdf[!, :PenaltyType] .= "ODE and adversarial penalty"
append!(all_prederrs_df, prederrdf)

Finally, we generate and save the plot of the fitted latent trajectories of  selected individuals.

In [ ]:
plot_ODE_and_adversarial_penalty = plot(
    plot_selected_ids_final(
        ODE_and_adversarial_penalty_models["m1"], ODE_and_adversarial_penalty_models["m2"], 
        mixeddata, args_joint, selected_ids, 
        colors_points = ["#3182bd" "#9ecae1"; "#e6550d" "#fdae6b"], 
        marker_sizes = [7, 5]
    ), 
    plot_title="ODE and adversarial penalty"
)
savefig(plot_ODE_and_adversarial_penalty, joinpath(save_plots_dir, "ODE_and_adversarial_penalty.pdf"))
plot_ODE_and_adversarial_penalty

As the final step, we save the overall dataframe containing all prediction errors to a CSV file.

In [ ]:
CSV.write(joinpath(save_eval_dir, "prediction_errors.csv"), all_prederrs_df)